In [1]:
# !pip freeze > requirements.txt
# !pip install -r requirements.txt

In [2]:
import os
import json
from datetime import datetime
import pytz
from collections import Counter

import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from torchvision import models, transforms
from tqdm import tqdm
import numpy as np
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torchsummary import summary

import warnings
warnings.filterwarnings("ignore")

# Preprocessing

In [3]:
class PathLabelProcessor:
    def __init__(self, 
                 base_path,
                 folder_name,
                 devices,
                 symptom):
        self.base_path = base_path
        self.folder_name = folder_name
        self.devices = devices
        self.symptom = symptom
        
        self.label_images()
      
    def find_folders_by_name(self):
        matching_folders = []

        for root, dirs, files in os.walk(self.base_path):
            for dir_name in dirs:
                if self.folder_name in dir_name:
                    folder_path = os.path.join(root, dir_name)
                    matching_folders.append(folder_path)

        for folder_path in matching_folders:
            print(folder_path)
            
        return matching_folders

    def find_image_json_pairs(self):
        image_paths = []
        json_paths = []

        for folder_path in self.find_folders_by_name():
            for root, dirs, files in os.walk(folder_path):
                for image_file in filter(lambda x: x.lower().endswith(('jpg', 'png')), files):
                    image_path = os.path.join(root, image_file)
                    json_file = f"{os.path.splitext(image_path)[0]}.json"
                    if os.path.isfile(json_file):
                        image_paths.append(image_path)
                        json_paths.append(json_file)

        print(f'Total images: {len(image_paths)}, Total JSON files: {len(json_paths)}')
        
        return image_paths, json_paths

    def label_images(self):
        self.labeled_image_paths = []

        for image_path, json_path in zip(*self.find_image_json_pairs()):
            with open(json_path) as f:
                data = json.load(f)

            if data['images']['meta']['device'] in self.devices:
                label = 0 if data['label']['label_disease_lv_3'] in self.symptom else 1
                self.labeled_image_paths.append((image_path, label))
        
        self.symptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 0))
        self.asymptomatic_count = len(Counter(item for item in self.labeled_image_paths if item[1] == 1))

        print(f'Total cases: {len(self.labeled_image_paths)}')
        print(f'Number of symptomatic cases: {self.symptomatic_count}, Number of asymptomatic cases: {self.asymptomatic_count}')

class ImageDataset(Dataset):
    def __init__(self, labeled_image_paths, transform):
        self.labeled_image_paths = labeled_image_paths
        self.transform = transform

    def __len__(self):
        return len(self.labeled_image_paths)

    def __getitem__(self, idx):
        image_path, label = self.labeled_image_paths[idx]

        image = Image.open(image_path)
        image = self.transform(image)
        label = torch.tensor([label], dtype=torch.float32)

        return image, label

class DataLoaderMaker:
    def __init__(self,
                 dataset,
                 batch_size,
                 train_ratio=0,
                 num_workers=None):
        self.dataset = dataset
        self.train_ratio = train_ratio
        self.batch_size = batch_size
        self.num_workers = num_workers

        if train_ratio:
            self.split_and_make_dataloader()
        else:
            self.dataloader = self.make_dataloader(self.dataset)

    def make_dataloader(self, dataset, shuffle=False):
        dataloader = DataLoader(dataset=dataset,
                                batch_size=self.batch_size,
                                shuffle=shuffle,
                                num_workers=self.num_workers,
                                pin_memory=True)
        
        self.inspect_data(dataloader)
        
        return dataloader

    def split_and_make_dataloader(self):
        train_size = int(len(self.dataset) * self.train_ratio)
        test_size = len(self.dataset) - train_size
        train_dataset, test_dataset = random_split(self.dataset, [train_size, test_size])

        self.train_loader = self.make_dataloader(train_dataset, shuffle=True)
        self.test_loader = self.make_dataloader(test_dataset, shuffle=True)

    def inspect_data(self, dataloader):
        print("Inspecting Data...")

        class_counts = {}
        for _, labels in dataloader:
            labels = labels.type(torch.int)
            for label in labels.tolist():
                label_tuple = tuple(label)
                class_counts[label_tuple] = class_counts.get(label_tuple, 0) + 1

        print("- Class Counts:")
        for class_label, count in class_counts.items():
            print(f"  Class {class_label}: {count} samples")

In [4]:
%%time
base_path = 'eye/Train'
folder_name = '일반'
devices = ['스마트폰', '일반카메라']
symptom = ['유', '상', '하', '초기', '비성숙', '성숙']

processor = PathLabelProcessor(base_path=base_path,
                                folder_name=folder_name,
                                devices=devices,
                                symptom=symptom)

labeled_image_paths = processor.labeled_image_paths

eye/Train/고양이/안구/일반
eye/Train/개/안구/일반
Total images: 199816, Total JSON files: 199816
Total cases: 98646
Number of symptomatic cases: 22339, Number of asymptomatic cases: 76307
CPU times: user 6.52 s, sys: 1.99 s, total: 8.51 s
Wall time: 8.52 s


In [5]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

CPU times: user 210 µs, sys: 86 µs, total: 296 µs
Wall time: 305 µs


In [6]:
%%time
batch_size = 96
num_workers = os.cpu_count()
train_ratio = 0.8

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers,
                              train_ratio=train_ratio)

Inspecting Data...


# Modeling

In [ ]:
class ModelTrainer:
    def __init__(self, 
                 model,
                 device,
                 train_dataloader,
                 valid_dataloader,
                 loss_fn,
                 optimizer,
                 scheduler):
        self.device = device
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.valid_dataloader = valid_dataloader
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_f1_score = 0.0
        korea = pytz.timezone('Asia/Seoul')
        now = datetime.now(korea)
        start_time = now.strftime('%Y%m%d-%H%M%S')
        self.name = f'{start_time}_symptoms.pth'
        self.writer = SummaryWriter(log_dir=f'runs/{self.name}')

    def calculate_f1_score(self, predicted, labels):
        return f1_score(labels, predicted, average='binary')

    def calculate_auc_roc(self, predicted, labels):
        return roc_auc_score(labels, predicted)

    def train_one_epoch(self, epoch, num_epochs):
        self.model.train()
        total_loss = 0.0
        correct = 0
        total = 0
    
        for step, (inputs, labels) in enumerate(tqdm(self.train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
            inputs, labels = inputs.to(self.device), labels.to(self.device)
    
            outputs = self.model(inputs)
    
            loss = self.loss_fn(outputs, labels)
            total_loss += loss.item()
    
            probs = torch.sigmoid(outputs)
            predicted = (probs > 0.5).int()
    
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
            loss.backward()
    
            self.optimizer.step()
            self.optimizer.zero_grad()
    
        self.scheduler.step()
        self.writer.add_scalar('LearningRate', self.scheduler.get_last_lr()[0], epoch)
    
        avg_loss = total_loss / len(self.train_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/train', avg_loss, epoch)        
        self.writer.add_scalar('Accuracy/train', accuracy, epoch)

    def eval_one_epoch(self, epoch):
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in self.valid_dataloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
        
                outputs = self.model(inputs)
        
                loss = self.loss_fn(outputs, labels)
                total_loss += loss.item()
        
                probs = torch.sigmoid(outputs)
                predicted = (probs > 0.5).int()
        
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        avg_loss = total_loss / len(self.valid_dataloader)
        accuracy = correct / total
        self.writer.add_scalar('Loss/valid', avg_loss, epoch)
        self.writer.add_scalar('Accuracy/valid', accuracy, epoch)
        
        current_f1_score = self.calculate_f1_score(np.array(all_predicted), np.array(all_labels))
        current_auc_roc = self.calculate_auc_roc(np.array(all_predicted), np.array(all_labels))
        
        self.writer.add_scalar('F1 Score/valid', current_f1_score, epoch)
        self.writer.add_scalar('AUC-ROC/valid', current_auc_roc, epoch)
        
        if current_f1_score > self.best_f1_score:
            self.best_f1_score = current_f1_score
            torch.save(self.model, self.name)

    def train(self, num_epochs):
        for epoch in range(num_epochs):
            self.train_one_epoch(epoch, num_epochs)
            self.eval_one_epoch(epoch)

In [ ]:
model = models.efficientnet_b1()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 1)
pos_weight = torch.tensor([processor.symptomatic_count / processor.asymptomatic_count], device=device)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.001)

trainer = ModelTrainer(model=model,
                       device=device,
                       train_dataloader=data_loader.train_loader,
                       valid_dataloader=data_loader.test_loader,
                       loss_fn=loss_fn,
                       optimizer=optimizer,
                       scheduler=scheduler)

In [ ]:
trainer.train(30)

Epoch 30/30: 100%|██████████| 823/823 [06:58<00:00,  1.96batch/s]


# Evaluation

In [27]:
class ModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        self.model = torch.load(path)
        self.model.to(self.device)
        print("Loaded Model Information:")
        summary(self.model, input_size=(3, 240, 240))

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                
                probs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(probs.max(dim=1).values.cpu().numpy())

        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions)
        recall = recall_score(labels, predictions)
        f1 = f1_score(labels, predictions)

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print("Evaluation Results:")
        print(f"Confusion Matrix:\n{cm}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Min Probability: {min_probs:.4f}")
        print(f"Max Probability: {max_probs:.4f}")
        print(f"Standard Deviation of Probabilities: {std_probs:.4f}")
        print(f"Mean Probability: {mean_probs:.4f}")

In [15]:
%%time
base_path = 'eye/Valid'
folder_name = '일반'
devices = ['스마트폰', '일반카메라']
symptom = ['유', '상', '하', '초기', '비성숙', '성숙']

processor = PathLabelProcessor(base_path=base_path,
                                folder_name=folder_name,
                                devices=devices,
                                symptom=symptom)

labeled_image_paths = processor.labeled_image_paths

eye/Valid/고양이/안구/일반
eye/Valid/개/안구/일반
Total images: 24976, Total JSON files: 24976
Total cases: 13808
Number of symptomatic cases: 3272, Number of asymptomatic cases: 10536
CPU times: user 893 ms, sys: 268 ms, total: 1.16 s
Wall time: 1.16 s


In [28]:
%%time
transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

batch_size = 32
num_workers = os.cpu_count()

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

path = '20231204-134936_symptoms.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class (0,): 22339 samples
  Class (1,): 76307 samples
Loaded Model Information:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 120, 120]             864
       BatchNorm2d-2         [-1, 32, 120, 120]              64
              SiLU-3         [-1, 32, 120, 120]               0
            Conv2d-4         [-1, 32, 120, 120]             288
       BatchNorm2d-5         [-1, 32, 120, 120]              64
              SiLU-6         [-1, 32, 120, 120]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 120, 120]   

100%|██████████| 3083/3083 [02:19<00:00, 22.06it/s]


Evaluation Results:
Confusion Matrix:
[[22339     0]
 [76307     0]]
Accuracy: 0.2265
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Min Probability: 1.0000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0000
Mean Probability: 1.0000
CPU times: user 3min 2s, sys: 40.9 s, total: 3min 43s
Wall time: 3min 16s


In [28]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 0],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class (0,): 3272 samples
Loaded Model Information:
EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True

100%|██████████| 103/103 [00:05<00:00, 17.50it/s]

Evaluation Results:
Confusion Matrix:
[[3272]]
Accuracy: 1.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Min Probability: 1.0000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0000
Mean Probability: 1.0000
CPU times: user 6.6 s, sys: 2.49 s, total: 9.09 s
Wall time: 8.92 s


In [29]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 1],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...


- Class Counts:
  Class (1,): 10536 samples
Loaded Model Information:
EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (sca

100%|██████████| 330/330 [00:16<00:00, 20.56it/s]


Evaluation Results:
Confusion Matrix:
[[    0     0]
 [10536     0]]
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Min Probability: 1.0000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0000
Mean Probability: 1.0000
CPU times: user 20.1 s, sys: 5.56 s, total: 25.7 s
Wall time: 23.1 s


In [17]:
class ModelTester:
    def __init__(self, path, device, dataloader):
        self.device = device
        self.dataloader = dataloader
        self.model = models.vgg16_bn(pretrained=True)
        self.load_model(path)
        self.evaluate()

    def load_model(self, path):
        # Create a new VGG model
        self.model = models.vgg16_bn(pretrained=True)
        
        # Extract the number of input features of the last fully connected layer
        nr_filters = self.model.classifier[0].in_features
        
        # Replace the last fully connected layer with a new one for binary classification
        self.model.classifier = nn.Linear(nr_filters, 1)
        
        # Load the state dict of the pre-trained weights
        state_dict = torch.load(path, map_location=torch.device("cpu"))
        
        # Load the state dict into the modified VGG model, skipping unnecessary keys
        model_dict = self.model.state_dict()
        state_dict = {k: v for k, v in state_dict.items() if k in model_dict}
        model_dict.update(state_dict)
        self.model.load_state_dict(model_dict)
        
        # Move the model to the specified device
        self.model = self.model.to(self.device)
        
        print("Loaded Model Information:")
        summary(self.model, input_size=(3, 240, 240))

    def classify(self):
        self.model.eval()
        predictions = []
        labels = []
        probabilities = []

        with torch.no_grad():
            for inputs, targets in tqdm(self.dataloader):
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                outputs = self.model(inputs)
                
                probs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())
                probabilities.extend(probs.max(dim=1).values.cpu().numpy())

        return predictions, labels, probabilities

    def calculate_prob_stats(self, probabilities):
        probabilities = np.array(probabilities)
        min_probs = np.min(probabilities)
        max_probs = np.max(probabilities)
        std_probs = np.std(probabilities)
        mean_probs = np.mean(probabilities)

        return min_probs, max_probs, std_probs, mean_probs

    def evaluate(self):
        predictions, labels, probabilities = self.classify()
        cm = confusion_matrix(labels, predictions)
        accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions)
        recall = recall_score(labels, predictions)
        f1 = f1_score(labels, predictions)

        min_probs, max_probs, std_probs, mean_probs = self.calculate_prob_stats(probabilities)

        print("Evaluation Results:")
        print(f"Confusion Matrix:\n{cm}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Min Probability: {min_probs:.4f}")
        print(f"Max Probability: {max_probs:.4f}")
        print(f"Standard Deviation of Probabilities: {std_probs:.4f}")
        print(f"Mean Probability: {mean_probs:.4f}")

In [23]:
%%time
dataset = ImageDataset(labeled_image_paths=labeled_image_paths,
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

path = 'pre_eye.pt'

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...


- Class Counts:
  Class (0,): 22339 samples
  Class (1,): 76307 samples
Loaded Model Information:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 240, 240]           1,792
       BatchNorm2d-2         [-1, 64, 240, 240]             128
              ReLU-3         [-1, 64, 240, 240]               0
            Conv2d-4         [-1, 64, 240, 240]          36,928
       BatchNorm2d-5         [-1, 64, 240, 240]             128
              ReLU-6         [-1, 64, 240, 240]               0
         MaxPool2d-7         [-1, 64, 120, 120]               0
            Conv2d-8        [-1, 128, 120, 120]          73,856
       BatchNorm2d-9        [-1, 128, 120, 120]             256
             ReLU-10        [-1, 128, 120, 120]               0
           Conv2d-11        [-1, 128, 120, 120]         147,584
      BatchNorm2d-12        [-1, 128, 120, 120]             256
     

100%|██████████| 1028/1028 [04:45<00:00,  3.61it/s]


Evaluation Results:
Confusion Matrix:
[[22339     0]
 [76307     0]]
Accuracy: 0.2265
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Min Probability: 1.0000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0000
Mean Probability: 1.0000
CPU times: user 5min 25s, sys: 34.2 s, total: 5min 59s
Wall time: 5min 42s


In [24]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 0],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class (0,): 22339 samples
Loaded Model Information:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 240, 240]           1,792
       BatchNorm2d-2         [-1, 64, 240, 240]             128
              ReLU-3         [-1, 64, 240, 240]               0
            Conv2d-4         [-1, 64, 240, 240]          36,928
       BatchNorm2d-5         [-1, 64, 240, 240]             128
              ReLU-6         [-1, 64, 240, 240]               0
         MaxPool2d-7         [-1, 64, 120, 120]               0
            Conv2d-8        [-1, 128, 120, 120]          73,856
       BatchNorm2d-9        [-1, 128, 120, 120]             256
             ReLU-10        [-1, 128, 120, 120]               0
           Conv2d-11        [-1, 128, 120, 120]         147,584
      BatchNorm2d-12        [-1, 128, 120, 120]             256
             R

100%|██████████| 233/233 [01:06<00:00,  3.50it/s]


Evaluation Results:
Confusion Matrix:
[[22339]]
Accuracy: 1.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Min Probability: 1.0000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0000
Mean Probability: 1.0000
CPU times: user 1min 17s, sys: 11.5 s, total: 1min 28s
Wall time: 1min 23s


In [25]:
%%time
dataset = ImageDataset(labeled_image_paths=[item for item in labeled_image_paths if item[1] == 1],
                       transform=transform)

data_loader = DataLoaderMaker(dataset=dataset,
                              batch_size=batch_size,
                              num_workers=num_workers)

ModelTester(path=path, device=device, dataloader=data_loader.dataloader)

Inspecting Data...
- Class Counts:
  Class (1,): 76307 samples
Loaded Model Information:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 240, 240]           1,792
       BatchNorm2d-2         [-1, 64, 240, 240]             128
              ReLU-3         [-1, 64, 240, 240]               0
            Conv2d-4         [-1, 64, 240, 240]          36,928
       BatchNorm2d-5         [-1, 64, 240, 240]             128
              ReLU-6         [-1, 64, 240, 240]               0
         MaxPool2d-7         [-1, 64, 120, 120]               0
            Conv2d-8        [-1, 128, 120, 120]          73,856
       BatchNorm2d-9        [-1, 128, 120, 120]             256
             ReLU-10        [-1, 128, 120, 120]               0
           Conv2d-11        [-1, 128, 120, 120]         147,584
      BatchNorm2d-12        [-1, 128, 120, 120]             256
             R

100%|██████████| 795/795 [03:40<00:00,  3.61it/s]


Evaluation Results:
Confusion Matrix:
[[    0     0]
 [76307     0]]
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Min Probability: 1.0000
Max Probability: 1.0000
Standard Deviation of Probabilities: 0.0000
Mean Probability: 1.0000
CPU times: user 4min 13s, sys: 26.9 s, total: 4min 40s
Wall time: 4min 26s


# Hyperparameter Tuning

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from torch.utils.data import DataLoader
from your_dataset_module import CustomDataset
from your_trainer_module import ModelTrainer

def train_tune(config):
    model = models.efficientnet_b1()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, 1)
    loss_fn = nn.BCEWithLogitsLoss()
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.01)
    
    trainer = ModelTrainer(model=model,
                           device=device,
                           train_dataloader=train_dataloader,
                           valid_dataloader=valid_dataloader,
                           loss_fn=loss_fn,
                           optimizer=optimizer,
                           scheduler=scheduler)
    
    num_epochs = 10
    for epoch in range(num_epochs):
        trainer.train_one_epoch(epoch, num_epochs)
        trainer.eval_one_epoch(epoch)

    return trainer.best_f1_score

search_space = {
    "lr": tune.loguniform(1e-5, 1e-1),
    "weight_decay": tune.loguniform(1e-6, 1e-2),
    "step_size": tune.choice([3, 5, 7]),
    "gamma": tune.choice([0.01, 0.1, 0.5]),
}

scheduler = ASHAScheduler(max_t=10, grace_period=1)

analysis = tune.run(
    train_tune,
    config=search_space,
    num_samples=10,
    scheduler=scheduler,
    metric="f1_score",
    mode="max",
)

best_hyperparameters = analysis.get_best_config(metric="f1_score", mode="max")
print("Best Hyperparameters:", best_hyperparameters)

In [ ]:
class ImageClassifier:
    def __init__(self, device, image_path, image_transform, model_weights_path, model):
        self.image_path = image_path
        self.image_transform = image_transform
        self.device = device
        self.model = model
        self.load_model_weights(model_weights_path)

    def preprocess_image(self):
        image = Image.open(self.image_path)
        input_data = self.image_transform(image).unsqueeze(0)
        input_data = input_data.to(self.device)
        return input_data

    def load_model_weights(self, weights_path):
        weights = torch.load(weights_path)
        self.model.load_state_dict(weights)
        self.model.to(self.device)

    def classify_image(self):
        self.model.eval()
        input_data = self.preprocess_image()

        with torch.no_grad():
            output = self.model(input_data)
        
        probabilities = torch.nn.functional.softmax(output, dim=1)
        predicted_prob, predicted_class = torch.max(probabilities, 1)
        
        class_label = "Disease" if predicted_class.item() == 0 else "Normal"
        class_prob = probabilities[0, predicted_class].item() * 100

        print(f"The image is predicted to be {class_label} with {class_prob:.2f}% probability.")
                
        return predicted_class.item(), class_prob

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_path = 'kIHjZsKQ6HdQ.jpg'
image_transform = transforms.Compose([transforms.Resize((240, 240)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

model_weights_path = 'best_model.pth'
model = models.efficientnet_b1()
num_classes = 2
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)

classifier = ImageClassifier(device=device,
                             image_path=image_path,
                             image_transform=image_transform,
                             model_weights_path=model_weights_path,
                             model=model)

predicted_class = classifier.classify_image()

In [ ]:
# import cv2
# import numpy as np
 
# ### homomorphic filter는 gray scale image에 대해서 밖에 안 되므로
# ### YUV color space로 converting한 뒤 Y에 대해 연산을 진행
# img = cv2.imread('file path')
# img_YUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)    
# y = img_YUV[::0]    
 
# rows = y.shape[0]    
# cols = y.shape[1]
 
# ### illumination elements와 reflectance elements를 분리하기 위해 log를 취함
# imgLog = np.log1p(np.array(y, dtype='float') / 255) # y값을 0~1사이로 조정한 뒤 log(x+1)
 
# ### frequency를 이미지로 나타내면 4분면에 대칭적으로 나타나므로 
# ### 4분면 중 하나에 이미지를 대응시키기 위해 row와 column을 2배씩 늘려줌
# M = 2*rows + 1
# N = 2*cols + 1
 
# ### gaussian mask 생성 sigma = 10
# sigma = 10
# (X, Y) = np.meshgrid(np.linspace(0, N-1, N), np.linspace(0, M-1, M)) # 0~N-1(and M-1) 까지 1단위로 space를 만듬
# Xc = np.ceil(N/2) # 올림 연산
# Yc = np.ceil(M/2)
# gaussianNumerator = (X - Xc)**2 + (Y - Yc)**2 # 가우시안 분자 생성
 
# ### low pass filter와 high pass filter 생성
# LPF = np.exp(-gaussianNumerator / (2*sigma*sigma))
# HPF = 1 - LPF
 
# ### LPF랑 HPF를 0이 가운데로 오도록iFFT함. 
# ### 사실 이 부분이 잘 이해가 안 가는데 plt로 이미지를 띄워보니 shuffling을 수행한 효과가 났음
# ### 에너지를 각 귀퉁이로 모아 줌
# LPF_shift = np.fft.ifftshift(LPF.copy())
# HPF_shift = np.fft.ifftshift(HPF.copy())
 
# ### Log를 씌운 이미지를 FFT해서 LPF와 HPF를 곱해 LF성분과 HF성분을 나눔
# img_FFT = np.fft.fft2(imgLog.copy(), (M, N))
# img_LF = np.real(np.fft.ifft2(img_FFT.copy() * LPF_shift, (M, N)) # low frequency 성분
# img_HF = np.real(np.fft.ifft2(img_FFT.copy() * HPF_shift, (M, N)) # high frequency 성분
 
# ### 각 LF, HF 성분에 scaling factor를 곱해주어 조명값과 반사값을 조절함
# gamma1 = 0.3
# gamma2 = 1.5
# img_adjusting = gamma1*img_LF[0:rows, 0:cols] + gamma2*img_HF[0:rows, 0:cols]
 
# ### 조정된 데이터를 이제 exp 연산을 통해 이미지로 만들어줌
# img_exp = np.expm1(img_adjusting) # exp(x) + 1
# img_exp = (img_exp - np.min(img_exp)) / (np.max(img_exp) - np.min(img_exp)) # 0~1사이로 정규화
# img_out = np.array(255*img_exp, dtype = 'uint8') # 255를 곱해서 intensity값을 만들어줌
 
# ### 마지막으로 YUV에서 Y space를 filtering된 이미지로 교체해주고 RGB space로 converting
# img_YUV[:,:,0] = img_out
# result = cv2.cvtColor(img_YUV, cv2.COLOR_YUV2BGR)
# cv2.imshow('homomorphic', result)